<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       BinarySeriesOp function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>BinarySeriesOp</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The BinarySeriesOp() function performs a point wise mathematical operation on two time series of equal size. The principal mathematical operation can be subtraction, addition, multiplication and division. It is called a point wise operation because it performs the mathematical operation one sample point at a time.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Common uses of BinarySeriesOp() are:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Subtracting trends from a time series to create a model from it.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Restoring trends to a time series before using the model for forecasting.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>As a building block to formulate more complex functions.</li></p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following procedure is an example of how to use BinarySeriesOp() to convolve two series with digital signal processing:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use DFFT() function on series 1 and series 2 to get dataframes named 'dfftRes1' and 'dfftRes2', respectively.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use BinarySeriesOp() to do point-wise multiplication using 'dfftRes1' and 'dfftRes2'.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use IDFFT() on the output of BinarySeriesOp() to get the convolved result of the two series.</li>
</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    BinarySeriesOp,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_BinarySeriesOp_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["binary_complex_left", "binary_complex_right"])

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data1 = DataFrame.from_table("binary_complex_left")
data2 = DataFrame.from_table("binary_complex_right")
data1 

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. BinarySeriesOp</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The BinarySeriesOp() function can be configured to operate in one of three input modes - ONE2ONE, MANY2ONE, and MATCH. These modes determine the number of primary series and number of secondary series involved in the operation, as well as determining how the primary and secondary series will be matched together.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The first step is to convert the data into TDSeries, which is required for the input time series which are passed to the BinarySeriesOp function.</p>


In [ ]:
data1_series_df = TDSeries(data=data1,
                               id="id",
                               row_index="seq",
                               row_index_style="SEQUENCE",
                               payload_field=["real_val", "imaginary_val"],
                               payload_content="COMPLEX") 

In [ ]:
data2_series_df = TDSeries(data=data2,
                               id="id",
                               row_index="seq",
                               row_index_style= "SEQUENCE",
                               payload_field=["real_val", "imaginary_val"],
                               payload_content="COMPLEX")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Than, we form the filter expressions to filter the series with id=1.</p>

In [ ]:
data1_filter_expr=data1_series_df.id==1
data2_filter_expr=data2_series_df.id==1

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We use the BinarySeriesOp to perform addition of two time series of equal size.</p>

In [ ]:
uaf_out = BinarySeriesOp(data1=data1_series_df,
                             data1_filter_expr=data1_filter_expr,
                             data2=data2_series_df,
                             data2_filter_expr=data2_filter_expr,
                             math_op="ADD",
                             input_fmt_input_mode="MANY2ONE")

binaryseries_df = uaf_out.result
binaryseries_df

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The result series always inherits the identifiers, series_id from the primary series.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables created above.</p>

In [ ]:
db_drop_table("binary_complex_left")
db_drop_table("binary_complex_right")

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>BinarySeriesOp function reference: <a href = 'https://docs.teradata.com/search/all?query=BinarySeriesOp&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>